In [2]:
import json
import pandas as pd
from collections import defaultdict
import os
from collections import Counter

In [3]:
pip install langchain langchain_community transformers SQLAlchemy==2.0.27 openai faiss-cpu

  Using cached langchain-0.2.0-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.2.0-py3-none-any.whl.metadata (8.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 633.7 kB/s eta 0:00:00 0:00:01
  Using cached SQLAlchemy-2.0.27-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached openai-1.30.1-py3-none-any.whl.metadata (21 kB)
  Using cached faiss_cpu-1.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.6 kB)
  Using cached aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached dataclasses_json-0.6.6-py3-none-any.whl.metadata (25 kB)
  Using cached langchain_core-0.2.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.2.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached huggingface_hub-0.23.1-py3-none-any.whl.metadata (12 kB)
  Using cached to

In [4]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain_community.llms.bedrock import Bedrock
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.embeddings import BedrockEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.chains import RetrievalQA

In [6]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
ls

'BLV EMR Data.csv'   csv_chat-Copy1.ipynb   csv_chat.ipynb    utils/
'EMR SCs Data.csv'   csv_chat-Copy2.ipynb   large_csv.ipynb


In [9]:
#Read The Data

In [10]:
ls

'BLV EMR Data.csv'   csv_chat-Copy1.ipynb   csv_chat.ipynb    utils/
'EMR SCs Data.csv'   csv_chat-Copy2.ipynb   large_csv.ipynb


In [11]:
pwd

'/root/large csv'

In [12]:
loader = CSVLoader(file_path="/root/large csv/BLV EMR Data.csv",encoding='ISO-8859-1')

data = loader.load()

In [13]:
len(data)

852820

In [14]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [15]:
inference_modifier = {
    "temperature": 0.9,
    "p": 0.9,
    "max_tokens": 4096
}

textgen_llm= Bedrock(
    model_id="cohere.command-text-v14",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

In [16]:
textgen_llm.invoke('tell me  a joke')

' Sure! Here is a joke for you:\n\nWhy did the banker break up with his girlfriend?\nBecause he could not lend her his heart.\n\nHow do you feel about this joke? Would you like to hear another one? '

In [17]:
summary_chain = load_summarize_chain(
    llm=textgen_llm,
    chain_type='map_reduce'
)

In [18]:
#docs

In [19]:
bedrock_embeddings = BedrockEmbeddings(model_id="cohere.embed-english-v3", client=boto3_bedrock)

In [ ]:
index = FAISS.from_documents(data, bedrock_embeddings)

In [ ]:
retriever = index.as_retriever()
retriever.search_kwargs['fetch_k'] = 20
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10
 
#llm = ChatOpenAI()
 
chain = RetrievalQA.from_chain_type(
    llm=textgen_llm, 
    retriever=retriever,
    chain_type='map_reduce',
    verbose=True
)

In [ ]:
2